In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
%matplotlib inline


In [2]:
import pandas as pd
dftrain=pd.read_csv("train.tsv", sep="\t")
dfdev=pd.read_csv("valid.tsv", sep="\t")
dftest=pd.read_csv("test.tsv", sep="\t")
dftrain1=pd.read_csv("trainquestiontype.csv")
dfdev1=pd.read_csv("validquestiontype.csv")
dftest1=pd.read_csv("testquestiontype.csv")

In [3]:
print(dftrain.shape)
print(dfdev.shape)
print(dftest.shape)
print(dftrain1.shape)
print(dfdev1.shape)
print(dftest1.shape)

(8588, 7)
(302, 7)
(230, 7)
(17176, 14)
(604, 14)
(460, 14)


In [4]:
ytrainlist=list(dftrain['is_duplicate'])
ydevlist=list(dfdev['is_duplicate'])
ytestlist=list(dftest['is_duplicate'])

In [5]:
Xtrain = dftrain1.drop('id', axis=1)
Xdev = dfdev1.drop('id', axis=1)
Xtest = dftest1.drop('id', axis=1)

In [6]:
xtrain=np.array(Xtrain)
xtrain1=xtrain[:int(xtrain.shape[0]/2)]
xtrain2=xtrain[int(xtrain.shape[0]/2):]
xtrain3=np.hstack((xtrain1,xtrain2))
xdev=np.array(Xdev)
xdev1=xdev[:int(xdev.shape[0]/2)]
xdev2=xdev[int(xdev.shape[0]/2):]
xdev3=np.hstack((xdev1,xdev2))
xtest=np.array(Xtest)
xtest1=xtest[:int(xtest.shape[0]/2)]
xtest2=xtest[int(xtest.shape[0]/2):]
xtest3=np.hstack((xtest1,xtest2))

In [7]:
X_train=np.vstack((xtrain3,xdev3))
X_train.shape
y_trainlist=ytrainlist+ydevlist
len(y_trainlist)
print(len(ytrainlist))
print(len(ydevlist))

8588
302


In [8]:
Xtrain5=np.copy(X_train)
for i in range(len(Xtrain5)):
    for j in range(26):
        if Xtrain5[i][j]<0.5:
            Xtrain5[i][j]=0
        else:
            Xtrain5[i][j]=1
Xtest5=np.copy(xtest3)
for i in range(len(Xtest5)):
    for j in range(26):
        if Xtest5[i][j]<0.5:
            Xtest5[i][j]=0
        else:
            Xtest5[i][j]=1
print(Xtrain5.shape)
print(Xtest5.shape)

(8890, 26)
(230, 26)


In [9]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

In [10]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

In [11]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Linear SVM": SVC(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=10000),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "Neural Net": MLPClassifier(alpha = 0.01,hidden_layer_sizes=(40, 20, 20), max_iter=50000),
    "Naive Bayes": GaussianNB()
}

In [12]:
# clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)
# clf.fit(X_train,np.array(y_trainlist))
# print(clf.score(X_train,np.array(y_trainlist)))
# print(clf.best_params_)
# result=clf.predict(xtest3)


In [13]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.clock()
        classifier.fit(X_train, Y_train)
        t_end = time.clock()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        Y_pred = classifier.predict(X_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff, 'predictions':Y_pred}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [14]:
print(Xtrain5.shape)
print(len(y_trainlist))
dict_models = batch_classify(Xtrain5, np.array(y_trainlist), Xtest5, np.array(ytestlist), no_classifiers = 11)
display_dict_models(dict_models)

(8890, 26)
8890
trained Logistic Regression in 0.04 s


/home/prakhar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


trained Nearest Neighbors in 0.59 s


/home/prakhar/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


trained Linear SVM in 4.53 s
trained Gradient Boosting Classifier in 32.12 s
trained Decision Tree in 0.01 s
trained Random Forest in 2.32 s
trained Neural Net in 5.63 s
trained Naive Bayes in 0.01 s


,classifier,train_score,test_score,train_time
3,Gradient Boosting Classifier,0.734308,0.682609,32.119305
4,Decision Tree,0.734308,0.678261,0.006898
5,Random Forest,0.734308,0.673913,2.315564
6,Neural Net,0.733971,0.669565,5.632212
1,Nearest Neighbors,0.533296,0.565217,0.589357
2,Linear SVM,0.670754,0.508696,4.526008
7,Naive Bayes,0.515073,0.452174,0.012175
0,Logistic Regression,0.590551,0.365217,0.036028


In [15]:
list2=np.array(dict_models['Gradient Boosting Classifier']['predictions'])
list1=np.arange(1,len(list2)+1)
list1=list1.reshape(-1,1)
list2=list2.reshape(-1,1)
list1=np.hstack((list1,list2))
import pandas as pd
from pandas import DataFrame
df=pd.DataFrame(list1)
df.columns = ['pair_id','label']
import csv
df.to_csv("submissions.csv")

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
y=dict_models['Gradient Boosting Classifier']['predictions']
print(confusion_matrix(y, ytestlist))
print(accuracy_score(y, ytestlist))

[[77 35]
 [38 80]]
0.6826086956521739
